In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = '../data/'

In [3]:
files = os.listdir(path)
files

['building_age_variables_railbelt.csv',
 'building_height_and_sqft_railbelt.csv',
 'building_height_railbelt.csv',
 'climate_reduced_AK_1981_2010.csv',
 'climate_reduced_railbelt_1981_2010.csv',
 'climate_reduced_railbelt_2000.csv',
 'climate_reduced_railbelt_2010.csv',
 'climate_reduced_railbelt_80.csv',
 'climate_reduced_railbelt_90.csv',
 'climate_reduced_railbelt_recent_1991_2020.csv']

In [4]:
# create dictionary of columns to keep for each data file
climate_keep = ['FD_sum_mean', 'TD_sum_mean', 'osm_id']

column_dict = {}
column_dict['climate_reduced_railbelt_1981_2010.csv'] = climate_keep
column_dict['climate_reduced_railbelt_2000.csv'] = climate_keep
column_dict['climate_reduced_railbelt_2010.csv'] = climate_keep
column_dict['climate_reduced_railbelt_90.csv'] = climate_keep
column_dict['climate_reduced_railbelt_recent_1991_2020.csv'] = climate_keep
column_dict['climate_reduced_railbelt_80.csv'] = climate_keep
column_dict['building_height_and_sqft_railbelt.csv'] = ['areasq_ft', 'height', 'osm_id', 'zip_code', 'zip_group']
column_dict['building_age_variables_railbelt.csv'] = ['age', 'osm_id']


In [5]:
def drop(csv_name, path, keep_cols):
    dataframe = pd.read_csv(path + csv_name)
    no_dups = dataframe.drop_duplicates('osm_id')
    cols_to_keep  = no_dups[keep_cols]   
    return cols_to_keep

In [6]:
subset_dataframes = []
for file in files:
    if file in column_dict.keys():
        subset_df = drop(file, path, column_dict[file])
        if file.startswith("climate_reduced"):
            col_suffix = file.split('_railbelt_')[1].strip('.csv')
            subset_df.rename(columns={ "FD_sum_mean": f"FD_{col_suffix}", "TD_sum_mean": f"TD_{col_suffix}" }, inplace = True)
        subset_dataframes.append(subset_df)


In [7]:
# check missing
for df in subset_dataframes:
    assert len(df['osm_id'].unique()) ==  len(df['osm_id'])
    print(df.isnull().sum()/ len(df))

age       0.0
osm_id    0.0
dtype: float64
areasq_ft    0.000000
height       0.000046
osm_id       0.000000
zip_code     0.000000
zip_group    0.000000
dtype: float64
FD_1981_2010    0.000009
TD_1981_2010    0.000009
osm_id          0.000000
dtype: float64
FD_2000    0.000009
TD_2000    0.000009
osm_id     0.000000
dtype: float64
FD_2010    0.000009
TD_2010    0.000009
osm_id     0.000000
dtype: float64
FD_80     0.000009
TD_80     0.000009
osm_id    0.000000
dtype: float64
FD_90     0.000009
TD_90     0.000009
osm_id    0.000000
dtype: float64
FD_recent_1991_2020    0.000009
TD_recent_1991_2020    0.000009
osm_id                 0.000000
dtype: float64


In [8]:
# joining dataframes

merged_df = subset_dataframes[files.index('building_age_variables_railbelt.csv')]
other_df = list(subset_dataframes)
other_df.pop(files.index('building_age_variables_railbelt.csv'))

for df in other_df:
    merged_df = pd.merge(merged_df, df, on = 'osm_id')
    

In [9]:
merged_df.head(5)

,age,osm_id,areasq_ft,height,zip_code,zip_group,FD_1981_2010,TD_1981_2010,FD_2000,TD_2000,FD_2010,TD_2010,FD_80,TD_80,FD_90,TD_90,FD_recent_1991_2020,TD_recent_1991_2020
0,2004.0,485655552,1011.819332,-1.156223,99507,9950,-992.202746,1847.074732,-941.482003,1945.692756,-765.664519,2051.646462,-998.157617,1769.070361,-1036.968618,1826.461078,-914.705047,1941.266765
1,1994.0,821173300,943.256095,0.272816,99686,9968,-1832.949579,814.575094,-1804.555087,890.371345,-1494.731646,901.032579,-1854.483655,760.098123,-1839.809995,793.255815,-1713.032242,861.553246
2,1997.0,821210009,825.083952,0.406550,99686,9968,-1832.949579,814.575094,-1804.555087,890.371345,-1494.731646,901.032579,-1854.483655,760.098123,-1839.809995,793.255815,-1713.032242,861.553246
3,1995.0,821210030,597.375132,0.529883,99686,9968,-1832.949579,814.575094,-1804.555087,890.371345,-1494.731646,901.032579,-1854.483655,760.098123,-1839.809995,793.255815,-1713.032242,861.553246
4,1994.0,821210032,307.360121,-0.293179,99686,9968,-1832.949579,814.575094,-1804.555087,890.371345,-1494.731646,901.032579,-1854.483655,760.098123,-1839.809995,793.255815,-1713.032242,861.553246


In [10]:
merged_df.to_csv('../output/joined_data_missing.csv')

In [11]:
print(merged_df.isnull().sum()/ len(merged_df))

age                    0.000000
osm_id                 0.000000
areasq_ft              0.000000
height                 0.000046
zip_code               0.000000
zip_group              0.000000
FD_1981_2010           0.000009
TD_1981_2010           0.000009
FD_2000                0.000009
TD_2000                0.000009
FD_2010                0.000009
TD_2010                0.000009
FD_80                  0.000009
TD_80                  0.000009
FD_90                  0.000009
TD_90                  0.000009
FD_recent_1991_2020    0.000009
TD_recent_1991_2020    0.000009
dtype: float64


In [12]:
merged_df.dropna(inplace=True)

merged_df.to_csv('../output/joined_data_nonmissing.csv')